In [6]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import sys
import math
from typing import List, Tuple
from collections import namedtuple
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
from itertools import product
from mip import *
import datetime
from random import shuffle
import networkx as nx

plt.rcParams['figure.figsize'] = [18, 13]

In [7]:
Point = namedtuple("Point", ['x', 'y'])

def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

def shift(seq, n=0):
    a = n % len(seq)
    return seq[-a:] + seq[:-a]

def sol_plot(solution,points):
    nodeCount=len(points)
    # plot the solution
    j=0
    for i in points:
        plt.plot(i.x, i.y,'ko')
        #plt.annotate(j,(i.x, i.y))
        j=j+1
    
    for i in range(-1, nodeCount-1):
        plt.plot((points[solution[i]].x, points[solution[i+1]].x),
                 (points[solution[i]].y, points[solution[i+1]].y),'-')
        
def objec(solution,points,d_m):
    nodeCount=len(points)
    obj = d_m[solution[-1]][solution[0]]
    for index in range(0, nodeCount-1):
        obj += d_m[solution[index]][solution[index+1]]
    return obj

def idx_two_opt(sol, idx1, idx2):
    # makes the reversal 
    if idx1<idx2:
        sol1=sol[:idx1]
        sol2=list(reversed(sol[idx1:idx2+1]))
        sol3=sol[idx2+1:]
    else:
        sol1=sol[:idx2]
        sol2=list(reversed(sol[idx2:idx1+1]))
        sol3=sol[idx1+1:]
    new_sol=sol1+sol2+sol3
    return new_sol
        
def two_opt_search(sol,points,d_m):
    nodeCount=len(points)
    best_sol=sol
    best_obj=objec(sol,points,d_m)
    for i in tqdm(range(nodeCount)):
            for j in range(nodeCount):
                p_sol=idx_two_opt(best_sol,i,j)
                p_obj=objec(p_sol,points,d_m)
                if p_obj<best_obj:
                    best_sol=p_sol
                    best_obj=p_obj
    return best_sol
        
def greedy_solve(points,s_node,k_nn):
    nodeCount=len(points)
    # visit the unvisited node that is closest to current node
    vis_list=[]
    # starts on random node
    #c_node=random.randint(0,nodeCount-1)
    c_node=s_node
    vis_list.append(c_node)
    for i in tqdm(range(nodeCount-1)):
        #checks available nodes
        av_nodes = [n for n in list(range(0,nodeCount)) if n not in vis_list]
        # first choose k_nn and check availability
        found=0
        # go through k_nn starting closest to c_node and check if available
        for i in k_nn[c_node]:
            if i in av_nodes:
                found=1
                next_node=i
                break
            else:
                next_node = min(av_nodes,key=lambda x:length(points[c_node],points[x]))
        # update current node        
        c_node=next_node
        vis_list.append(c_node) 
    solution = vis_list
    return solution

        
class SubTourCutGenerator(ConstrsGenerator):
    """Class to generate cutting planes for the TSP"""
    def __init__(self, Fl: List[Tuple[int, int]], x_, V_):
        self.F, self.x, self.V = Fl, x_, V_

    def generate_constrs(self, model: Model, depth: int = 0, npass: int = 0):
        xf, V_, cp, G = model.translate(self.x), self.V, CutPool(), nx.DiGraph()
        for (u, v) in [(k, l) for (k, l) in product(V_, V_) if k != l and xf[k][l]]:
            G.add_edge(u, v, capacity=xf[u][v].x)
        for (u, v) in self.F:
            val, (S, NS) = nx.minimum_cut(G, u, v)
            if val <= 0.99:
                aInS = [(xf[i][j], xf[i][j].x)
                        for (i, j) in product(V_, V_) if i != j and xf[i][j] and i in S and j in S]
                if sum(f for v, f in aInS) >= (len(S)-1)+1e-4:
                    cut = xsum(1.0*v for v, fm in aInS) <= len(S)-1
                    cp.add(cut)
                    if len(cp.cuts) > 256:
                        for cut in cp.cuts:
                            model += cut
                        return
        for cut in cp.cuts:
            model += cut        

def tsp_mip_solver(input_data):
    # parse the input
    lines = input_data.split('\n')

    nodeCount = int(lines[0])

    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))
    print('Points parsed!')
    
    # calculate distance matrix
    d_m = [ [length(q,p) for q in points] for p in points]
    print('Distance matrix ready!')
    
    # declare MIP model
    m = Model(solver_name='GRB')
    print('-Model instatiated!',datetime.datetime.now())
    
    # states search emphasis
    #     - '0' (default) balanced approach
    #     - '1' (feasibility) aggressively searches for feasible solutions
    #     - '2' (optimality) explores search space to tighten dual gap
    m.emphasis = 0
    
    # whenever the distance of the lower and upper bounds is less or 
    # equal max_gap*100%, the search can be finished
    m.max_mip_gap = 0.05
    
    # specifies number of used threads
    # 0 uses solver default configuration, 
    # -1 uses the number of available processing cores 
    # ≥1 uses the specified number of threads. 
    # An increased number of threads may improve the solution time but also increases 
    # the memory consumption. Each thread needs to store a different model instance!
    m.threads = -1
    
    
    # controls the generation of cutting planes
    # cutting planes usually improve the LP relaxation bound but also make the solution time of the LP relaxation larger
    # -1 means automatic
    #  0 disables completely
    #  1 (default) generates cutting planes in a moderate way
    #  2 generates cutting planes aggressively
    #  3 generates even more cutting planes
    m.cuts=3
    
    m.preprocess=1
    m.pump_passes=10
    #m.sol_pool_size=1
    
    # defining nodes as a set
    nodes = set(range(nodeCount))
    
    # defining possible transitions from node to node
    arcs = [(i, j) for (i, j) in product(nodes, nodes) if i != j]
    
    
    # instantiate "entering and leaving" variables
    x = [ [m.add_var(name="x{}_{}".format(p,q),var_type='B') for q in nodes] for p in nodes ] 
    # instantiate subtour elimination variables 
    y = [ m.add_var(name="y{}".format(i)) for i in nodes ]
    print('-Variables instantiated',datetime.datetime.now())
    
    # declare objective function
    m.objective = minimize( xsum( d_m[i][j]*x[i][j] for i in nodes for j in nodes) )
    
    print('-Objective declared!',datetime.datetime.now())
    
    # declare constraints
    # leave each city only once
    for i in nodes:
        m.add_constr( xsum(x[i][j] for j in nodes - {i}) == 1 )

    # enter each city only once
    for i in nodes:
        m.add_constr( xsum(x[j][i] for j in nodes - {i}) == 1 )
    
    # weak subtour elimination constraints
    for (i, j) in tqdm(product(nodes - {0}, nodes - {0})):
        if i != j:
            m.add_constr( y[i] - (nodeCount+1)*x[i][j] >= y[j]-nodeCount )
            
    # size 2 subtour elimination constraints
    for (i, j) in arcs:
            m.add_constr( x[i][j] + x[j][i] <= 1 )
            
            
            
    print('-Constraints declared!',datetime.datetime.now())
    
    #Maximum time in seconds that the search can go on if a feasible solution 
    #is available and it is not being improved
    mssi = 100 #default = inf
    # specifies maximum number of nodes to be explored in the search tree (default = inf)
    mn = 100000 #default = 1073741824
    # optimize model m within a processing time limit of 'ms' seconds
    ms = 300 #default = inf
    
    S=[i for i in range(nodeCount)]
    n_it = 5
    run_twoopt = True
    thld = 0.99
    obj = objec(S,points,d_m)
    if run_twoopt == True:
        for i in range(n_it):
            print('Two opt it ({}/{})'.format(i+1,n_it))
            s_obj = obj
            S = two_opt_search(S,points,d_m)
            obj=objec(S,points,d_m)
            print('S obj = {} / F obj = {}'.format(s_obj,obj))
            if obj/s_obj >= thld:
                print('No significant improvement, breaking.')
                break
    
    sol_plot(S,points)
    
    # warm start with best 2-opt solution
    m.start = [(x[S[k-1]][S[k]], 1.0) for k in range(nodeCount)]
    print('Warm start set up.')
    
    # computing farthest point for each point, these will be checked first for
    # isolated subtours
    print('Preprocessing nodes for Constr. Gen.')
    F, G = [], nx.DiGraph()
    for (i, j) in tqdm(arcs):
        G.add_edge(i, j, weight=d_m[i][j])
    for i in tqdm(nodes):
        P, D = nx.dijkstra_predecessor_and_distance(G, source=i)
        DS = list(D.items())
        DS.sort(key=lambda x: x[1])
        F.append((i, DS[-1][0]))
        
    #print('Setting up Subtour Cut Generator')
    # sets up the cut generator
    #m.cuts_generator = SubTourCutGenerator(F, x, nodes)

    # do a really short optimization to set up the variables
    print('Setting up LNS',datetime.datetime.now() )
    #status = m.optimize(max_seconds = 10)
    
    sol = S
    start_obj = objec(S,points,d_m)
    alternate = False
    n_it=10
    free_vars_n=100
    #print('sol:',sol)
    for i in range(n_it):
        print('\n it ({}/{}) | Current obj = {}'.format(i+1,n_it,objec(sol,points,d_m)))
        # instantiates a copy model for the LNS iteration
        print('Copying model...')
        m2 = m.copy(solver_name='GRB')
        m2.start = [(x[sol[k-1]][sol[k]], 1.0) for k in range(nodeCount)]
        # fix a (random?) subset of variables
        if alternate == True:
            if i%2 == 0:
                idx_fixed_vars = random.sample(list(enumerate(sol)), nodeCount-free_vars_n)
                print('Disjoint variable set fixed.')
            else:
                placeholder = shift(list(enumerate(sol)), random.randrange(nodeCount) )
                idx_fixed_vars = placeholder[0:nodeCount-free_vars_n]
                print('Contiguous variable set fixed.')
        else:
            placeholder = shift(list(enumerate(sol)), random.randrange(nodeCount) )
            idx_fixed_vars = placeholder[0:nodeCount-free_vars_n]
                
            
        print('Fixed {} nodes'.format(len(idx_fixed_vars)))
        # add LNS equality constraints
        LNS_constr = []
        for i in tqdm(range(len(idx_fixed_vars))):
            for n in nodes:
                    if sol[idx_fixed_vars[i][0]] == n:
                        LNS_constr.append(m2.add_constr( x[sol[idx_fixed_vars[i][0]-1]][n] == 1,
                                          name="LNS_x{}_{}".format(sol[idx_fixed_vars[i][0]-1],n)))
                    else:
                        LNS_constr.append(m2.add_constr( x[sol[idx_fixed_vars[i][0]-1]][n] == 0,
                                          name="LNS_x{}_{}".format(sol[idx_fixed_vars[i][0]-1],n)))
        # optimize
        print('Optimizing neighborhood. Costr vec size: {}'.format(len(LNS_constr)))
        m2.clique=2
        m2.preprocess=1
        m2.cuts=3
        #m2.max_mip_gap = 1
        m2.emphasis = 2
        m2.clique_merge()
        m2.cuts_generator = SubTourCutGenerator(F, x, nodes)
        status = m2.optimize(max_seconds = ms)       
        # extract resulting tour
        sol= [0]
        c_node=0
        for j in range(nodeCount-1):
            for i in range(nodeCount):
                if round(m2.var_by_name("x{}_{}".format(c_node,i)).x) != 0:
                    sol.append(i)
                    c_node=i
                    break
        #print('sol:',sol)
    
    # run short optimization to update objective and dual gap
    m2 = m.copy(solver_name='GRB')
    m2.start = [(x[sol[k-1]][sol[k]], 1.0) for k in range(nodeCount)]
    m2.clique_merge()
    status = m2.optimize(max_seconds = 20)
    
    print('Opt. Status:',status)
    print('Dual Bound:',m2.objective_bound)
    print('Dual gap:',m2.gap)
    
    print('\n Ended LNS. Starting Obj = {} | Final Obj = {}'.format(start_obj,objec(sol,points,d_m)))
    
    
    sol_plot(sol,points)
    obj=objec(sol,points,d_m)
    
    # prepare the solution in the specified output format
    if status == OptimizationStatus.OPTIMAL:
        output_data = '%.2f' % obj + ' ' + str(1) + '\n'
        output_data += ' '.join(map(str, sol))
    elif status == OptimizationStatus.FEASIBLE:
        output_data = '%.2f' % obj + ' ' + str(0) + '\n'
        output_data += ' '.join(map(str, sol))

    return output_data

In [ ]:
import os
notebook_path = os.path.abspath("Notebook.ipynb")
data_path = os.path.join(os.path.dirname(notebook_path), "data\\tsp_439_1")

if len(data_path) > 1:
    file_location = data_path.strip()
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
        print(tsp_mip_solver(input_data))


Points parsed!
Distance matrix ready!
-Model instatiated! 2020-08-11 21:21:06.121270
-Variables instantiated 2020-08-11 21:21:07.339136
-Objective declared! 2020-08-11 21:21:09.490356



-Constraints declared! 2020-08-11 21:21:28.752986
Two opt it (1/5)



S obj = 270651.6048375016 / F obj = 123168.30910223635
Two opt it (2/5)



S obj = 123168.30910223635 / F obj = 119797.14802261263
Two opt it (3/5)



S obj = 119797.14802261263 / F obj = 119143.38711929119
No significant improvement, breaking.
Warm start set up.
Preprocessing nodes for Constr. Gen.



Setting up LNS 2020-08-11 21:23:34.803453

 it (1/10) | Current obj = 119143.38711929119
Copying model...
